# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm, trange
from torch import optim
from torch.autograd import Variable
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [28]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer_1 = torch.nn.Sequential(torch.nn.Conv2d(1, 64, kernel_size = 3, stride = 1, padding = 1),
                                        torch.nn.ReLU(),
                                        torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
                                        torch.nn.ReLU(),
                                        torch.nn.MaxPool2d(stride = 2, kernel_size = 2))
        self.layer_2 = torch.nn.Sequential(torch.nn.Linear(14 * 14  * 128, 1024),
                                        torch.nn.ReLU(),
                                        torch.nn.Dropout(p = 0.5),
                                        torch.nn.Linear(1024, 10))
    
    def forward(self, x):
        x = self.layer_1(x)
        x = x.view(-1, 14 * 14 * 128)
        x = self.layer_2(x)
        return x
    
model = SimpleNet()

# TODO:define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.003)

Next, we can start to train and evaluate!

In [29]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_correct, train_total = 0, len(train_dataset)
    test_correct, test_total = 0, len(test_dataset)
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        X_train, y_train = Variable(images), Variable(labels)
        optimizer.zero_grad()
        outputs = model(X_train)
        _, y_pred = torch.max(outputs, 1)
        train_correct += torch.sum(y_pred == y_train.data)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    for images, labels in tqdm(test_loader):
        X_test, y_test = Variable(images), Variable(labels)
        outputs = model(X_test)
        _, y_pred = torch.max(outputs, 1)
        test_correct += torch.sum(y_pred == y_test.data)
    
    print("Training Accuracy: %.2f%%" % (train_correct.numpy() / train_total * 100))
    print("Testing Accuracy: %.2f%%" % (test_correct.numpy() / test_total * 100))

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.43it/s]


Training Accuracy: 95.06%
Testing Accuracy: 97.99%


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.41it/s]


Training Accuracy: 98.10%
Testing Accuracy: 97.85%


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.41it/s]


...

Training Accuracy: 99.23%
Testing Accuracy: 97.22%


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.43it/s]


Training Accuracy: 99.27%
Testing Accuracy: 98.42%


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:23<00:00,  3.36it/s]


Training Accuracy: 99.31%
Testing Accuracy: 98.46%


#### Q5:
Please print the training and testing accuracy.

In [30]:
model.eval()
train_correct, train_total = 0, len(train_dataset)
test_correct, test_total = 0, len(test_dataset)

for images, labels in tqdm(train_loader):
    X_train, y_train = Variable(images), Variable(labels)
    outputs = model(X_train)
    _, y_pred = torch.max(outputs, 1)
    train_correct += torch.sum(y_pred == y_train.data)

for images, labels in tqdm(test_loader):
    X_test, y_test = Variable(images), Variable(labels)
    outputs = model(X_test)
    _, y_pred = torch.max(outputs, 1)
    test_correct += torch.sum(y_pred == y_test.data)
    
print("Training Accuracy: %.2f%%" % (train_correct.numpy() / train_total * 100))
print("Testing Accuracy: %.2f%%" % (test_correct.numpy() / test_total * 100))

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:24<00:00,  3.19it/s]


Training Accuracy: 99.70%
Testing Accuracy: 98.76%
